**Dual LSTM Encoder for Dialog Response Generation**

http://www.wildml.com/2016/07/deep-learning-for-chatbots-2-retrieval-based-model-tensorflow/

https://github.com/dennybritz/chatbot-retrieval

https://github.com/rkadlec/ubuntu-ranking-dataset-creator

https://arxiv.org/abs/1506.08909

In [1]:
import tensorflow as tf
tf.VERSION

'1.2.0'

**Estimator**

https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/estimator/Estimator

https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/estimator

**model_fn**
```text
 |        model_fn: Model function. Follows the signature:
 |      
 |          * Args:
 |      
 |            * `features`: This is the first item returned from the `input_fn`
 |                   passed to `train`, 'evaluate`, and `predict`. This should be a
 |                   single `Tensor` or `dict` of same.
 |            * `labels`: This is the second item returned from the `input_fn`
 |                   passed to `train`, 'evaluate`, and `predict`. This should be a
 |                   single `Tensor` or `dict` of same (for multi-head models). If
 |                   mode is `ModeKeys.PREDICT`, `labels=None` will be passed. If
 |                   the `model_fn`'s signature does not accept `mode`, the
 |                   `model_fn` must still be able to handle `labels=None`.
 |            * `mode`: Optional. Specifies if this training, evaluation or
 |                   prediction. See `ModeKeys`.
 |            * `params`: Optional `dict` of hyperparameters.  Will receive what
 |                   is passed to Estimator in `params` parameter. This allows
 |                   to configure Estimators from hyper parameter tuning.
 |            * `config`: Optional configuration object. Will receive what is passed
 |                   to Estimator in `config` parameter, or the default `config`.
 |                   Allows updating things in your model_fn based on configuration
 |                   such as `num_ps_replicas`, or `model_dir`.
 |      
 |          * Returns:
 |            `EstimatorSpec`
 ```

In [2]:
def dual_encoder(vocab_size,
                 embed_size,
                 hidden_size,
                 input_context,
                 input_context_len,
                 input_utterance,
                 input_utterance_len,
                 targets):

    input_data = tf.concat([input_context, input_utterance], axis=0)
    input_length = tf.concat([input_context_len, input_utterance_len], axis=0)
    input_length = tf.reshape(input_length, [-1])
    
    embeddings = tf.get_variable(
        'embeddings',
        shape=(vocab_size, embed_size),
        initializer=tf.random_uniform_initializer(-0.25, 0.25))

    input_embed = tf.nn.embedding_lookup(
        embeddings, input_data, name='input_embed')
        
    with tf.variable_scope('rnn'):
        cell = tf.nn.rnn_cell.LSTMCell(
            hidden_size,
            forget_bias=2.0,
            use_peepholes=True,
            state_is_tuple=True)

        outputs, states = tf.nn.dynamic_rnn(
            cell,
            input_embed,
            sequence_length=input_length,
            dtype=tf.float32)

        context_encoding, utterance_encoding = tf.split(
            states.h, num_or_size_splits=2, axis=0)

    with tf.variable_scope('prediction'):
        ct = context_encoding
        rt = utterance_encoding
        M = tf.get_variable(
            'M',
            shape=(hidden_size, hidden_size),
            initializer=tf.truncated_normal_initializer())

        ct_M = tf.matmul(ct, M)
        batch_ct_M = tf.expand_dims(ct_M, axis=2)
        batch_rt = tf.expand_dims(rt, axis=2)
        batch_ct_M_r = tf.matmul(batch_ct_M, batch_rt, transpose_a=True)
        ct_M_r = tf.squeeze(batch_ct_M_r, axis=2)

        b = tf.get_variable(
           'b', shape=(), initializer=tf.zeros_initializer())
        
        logits = ct_M_r + b
        
        probs = tf.sigmoid(logits)

    if targets is None:
        return probs, None

    loss = tf.losses.sigmoid_cross_entropy(
        multi_class_labels=targets, logits=logits, reduction=tf.losses.Reduction.MEAN)
    
    return probs, loss

In [3]:
def model_fn_train(features, labels, vocab_size, embed_size, hidden_size, learning_rate, optimizer):
    input_context = features['context']
    input_context_len = features['context_len']
    input_utterance = features['utterance']
    input_utterance_len = features['utterance_len']

    probs, loss = dual_encoder(
        vocab_size,
        embed_size,
        hidden_size,
        input_context,
        input_context_len,
        input_utterance,
        input_utterance_len,
        labels)
    
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=learning_rate,
        clip_gradients=10.0,
        optimizer=optimizer)
    
    return tf.estimator.EstimatorSpec(
        mode=tf.estimator.ModeKeys.TRAIN,
        predictions=probs,
        loss=loss,
        train_op=train_op)

def model_fn_eval(features, labels, vocab_size, embed_size, hidden_size):
    input_context = []
    input_context_len = []
    input_utterance = []
    input_utterance_len = []
    labels_ = []
    
    context = features['context']
    context_len = features['context_len']

    input_context.append(context)
    input_context_len.append(context_len)
    input_utterance.append(features['utterance'])
    input_utterance_len.append(features['utterance_len'])
    labels_.append(tf.ones_like(context_len))
    
    for i in range(9):
        input_context.append(context)
        input_context_len.append(context_len)
        input_utterance.append(features['distractor_{}'.format(i)])
        input_utterance_len.append(features['distractor_{}_len'.format(i)])
        labels_.append(tf.zeros_like(context_len))

    input_context = tf.concat(input_context, axis=0)
    input_context_len = tf.concat(input_context_len, axis=0)
    input_utterance = tf.concat(input_utterance, axis=0)
    input_utterance_len = tf.concat(input_utterance_len, axis=0)
    labels_ = tf.concat(labels_, axis=0)

    probs, loss = dual_encoder(
        vocab_size,
        embed_size,
        hidden_size,
        input_context,
        input_context_len,
        input_utterance,
        input_utterance_len,
        labels_)
    
    split_probs = tf.split(probs, num_or_size_splits=10, axis=0)
    predictions = tf.concat(split_probs, axis=1)
    predictions_2 = predictions[:, :2]
    
    recall_at_1_2 = tf.metrics.recall_at_k(labels=labels, predictions=predictions_2, k=1)
    recall_at_1_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=1)
    recall_at_2_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=2)
    recall_at_5_10 = tf.metrics.recall_at_k(labels=labels, predictions=predictions, k=5)
    
    eval_metric_ops = {
        'recall_at_1_2': recall_at_1_2,
        'recall_at_1_10': recall_at_1_10,
        'recall_at_2_10': recall_at_2_10,
        'recall_at_5_10': recall_at_5_10,
    }

    return tf.estimator.EstimatorSpec(
        mode=tf.estimator.ModeKeys.EVAL,
        predictions=probs,
        loss=loss,
        train_op=None,
        eval_metric_ops=eval_metric_ops
    )

def model_fn(features, labels, mode, params):
    vocab_size = params['vocab_size']
    embed_size = params['embed_size']
    hidden_size = params['hidden_size']

    if mode == tf.estimator.ModeKeys.TRAIN:
        learning_rate = params['learning_rate']
        optimizer =  params['optimizer']
        return model_fn_train(
            features, labels, vocab_size, embed_size, hidden_size, learning_rate, optimizer)
    if mode == tf.estimator.ModeKeys.EVAL:
        return model_fn_eval(features, labels, vocab_size, embed_size, hidden_size)
    
    return None

**Input**

In [4]:
# `tokenizer` function must be defined before restoring the vocabulary object
# (pickle does not serialize functions)
def tokenizer(sentences):
    return (sentence.split() for sentence in sentences)

class VocabularyAdapter:
    
    def __init__(self, vocabulary_bin):
        self._vocab = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(vocabulary_bin)
    
    @property
    def size(self):
        return len(self._vocab.vocabulary_)
    
    @property
    def vector_length(self):
        return self._vocab.max_document_length


def features_train(vector_length):
    return [
        tf.feature_column.numeric_column(
            key='context', shape=vector_length, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='context_len', shape=1, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='utterance', shape=vector_length, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='utterance_len', shape=1, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='label', shape=1, dtype=tf.int64),
    ]

def features_eval(vector_length):
    features = []
    keys = ['context', 'utterance']
    keys += ['distractor_{}'.format(i) for i in range(9)]
    for key in keys:
        features += [
            tf.feature_column.numeric_column(
                key=key, shape=vector_length, dtype=tf.int64),
            tf.feature_column.numeric_column(
                key=key + '_len', shape=1, dtype=tf.int64),
        ]
    return features

def _input_reader(name, filenames, features, batch_size, num_epochs):
    example_features = tf.feature_column.make_parse_example_spec(features)
    return tf.contrib.learn.read_batch_record_features(
        file_pattern=filenames,
        features=example_features,
        batch_size=batch_size,
        num_epochs=num_epochs,
        randomize_input=True,
        queue_capacity=200000 + batch_size * 10,
        name='read_batch_record_features_' + name
    )


def input_train(name, filenames, features, batch_size, num_epochs=None):
    batch_example = _input_reader(name, filenames, features, batch_size, num_epochs)
    batch_target = batch_example.pop('label')
    return batch_example, batch_target

def input_eval(name, filenames, features, batch_size, num_epochs=None):
    batch_example = _input_reader(name, filenames, features, batch_size, num_epochs)
    batch_target = tf.zeros_like(batch_example['context_len'])
    return batch_example, batch_target

**Training**

In [5]:
import os

HOME_DIR = 'ubuntu'
DATA_DIR = os.path.join(HOME_DIR, 'data')
VOCAB_BIN = os.path.join(DATA_DIR, 'vocabulary.bin')
TRAIN_TFR = os.path.join(DATA_DIR, 'train.tfrecords')
VALID_TFR = os.path.join(DATA_DIR, 'valid.tfrecords')
TEST_TFR = os.path.join(DATA_DIR, 'test.tfrecords')

def has_file(file):
    if not os.path.isfile(file):
        raise Exception('File not found: {}'.format(file))

has_file(VOCAB_BIN)
has_file(TRAIN_TFR)
has_file(VALID_TFR)
has_file(TEST_TFR)

In [6]:
vocab = VocabularyAdapter(VOCAB_BIN)
train_features = features_train(vocab.vector_length)
eval_features = features_eval(vocab.vector_length)

In [7]:
params = {
    'vocab_size': vocab.size,
    'embed_size': 100,
    'hidden_size': 200,
    'learning_rate': 0.001,
    'optimizer': 'Adam',
    'batch_size': 256,
    'num_epochs': 10,
}

input_fn_train = lambda: input_train('train', [TRAIN_TFR], train_features, params['batch_size'], 1)
input_fn_valid = lambda: input_eval('valid', [VALID_TFR], eval_features, 16, 1)
input_fn_test = lambda: input_eval('test', [TEST_TFR], eval_features, 16, 1)

In [8]:
import shutil

def remove_dir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)

MODEL_DIR = os.path.join(HOME_DIR, 'model')

remove_dir(MODEL_DIR)

In [9]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=MODEL_DIR,
    params=params)

estimator

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'ubuntu/model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}


In [10]:
def epoch_range(i, num_epochs):
    range_start = i * num_epochs + 1
    range_end = range_start + num_epochs
    return range(range_start, range_end)

In [11]:
%%time

for epoch in epoch_range(0, params['num_epochs']):
    print('[ Epoch {} ]\n'.format(epoch))
    print('Training...\n')
    %time estimator.train(input_fn_train)
    print()
    print('Validation...\n')
    %time estimator.evaluate(input_fn_valid, name='valid')
    print()

[ Epoch 1 ]

Training...

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ubuntu/model/model.ckpt.
INFO:tensorflow:loss = 0.871889, step = 1
INFO:tensorflow:global_step/sec: 0.683466
INFO:tensorflow:loss = 0.70301, step = 101 (146.315 sec)
INFO:tensorflow:global_step/sec: 0.684033
INFO:tensorflow:loss = 0.690594, step = 201 (146.192 sec)
INFO:tensorflow:global_step/sec: 0.683925
INFO:tensorflow:loss = 0.679936, step = 301 (146.215 sec)
INFO:tensorflow:global_step/sec: 0.683901
INFO:tensorflow:loss = 0.663239, step = 401 (146.220 sec)
INFO:tensorflow:Saving checkpoints for 412 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.682968
INFO:tensorflow:loss = 0.661207, step = 501 (146.420 sec)
INFO:tensorflow:global_step/sec: 0.684255
INFO:tensorflow:loss = 0.655156, step = 601 

INFO:tensorflow:global_step/sec: 0.683758
INFO:tensorflow:loss = 0.433627, step = 5708 (146.250 sec)
INFO:tensorflow:global_step/sec: 0.683563
INFO:tensorflow:loss = 0.480067, step = 5808 (146.292 sec)
INFO:tensorflow:global_step/sec: 0.683698
INFO:tensorflow:loss = 0.462682, step = 5908 (146.264 sec)
INFO:tensorflow:Saving checkpoints for 5963 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.682661
INFO:tensorflow:loss = 0.413626, step = 6008 (146.486 sec)
INFO:tensorflow:global_step/sec: 0.684012
INFO:tensorflow:loss = 0.504914, step = 6108 (146.195 sec)
INFO:tensorflow:global_step/sec: 0.684115
INFO:tensorflow:loss = 0.45957, step = 6208 (146.175 sec)
INFO:tensorflow:global_step/sec: 0.684051
INFO:tensorflow:loss = 0.430668, step = 6308 (146.187 sec)
INFO:tensorflow:Saving checkpoints for 6374 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.682864
INFO:tensorflow:loss = 0.45897, step = 6408 (146.442 sec)
INFO:tensorflow:global_step/sec: 0.683956
INFO

INFO:tensorflow:loss = 0.388466, step = 11515 (146.495 sec)
INFO:tensorflow:global_step/sec: 0.683967
INFO:tensorflow:loss = 0.330176, step = 11615 (146.207 sec)
INFO:tensorflow:global_step/sec: 0.684051
INFO:tensorflow:loss = 0.333756, step = 11715 (146.187 sec)
INFO:tensorflow:Saving checkpoints for 11721 into ubuntu/model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.18051.
CPU times: user 1h 20min 26s, sys: 18min 13s, total: 1h 38min 40s
Wall time: 1h 35min 18s

Validation...

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Starting evaluation at 2017-06-30-02:41:15
INFO:tensorflow:Restoring parameters from ubuntu/model/model.ckpt-11721
INFO:tensorflow:Finished evaluation at 2017-06-30-02:47:36
INFO:tensorflow:Saving dict for global step 11721: global_step = 11721, loss = 0.544804, recall_at_1_10 = 0.48726993865, recall_at_1_2 = 0.841155419223, recall_

INFO:tensorflow:loss = 0.227112, step = 16329 (146.175 sec)
INFO:tensorflow:global_step/sec: 0.684183
INFO:tensorflow:loss = 0.262252, step = 16429 (146.159 sec)
INFO:tensorflow:Saving checkpoints for 16451 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.682989
INFO:tensorflow:loss = 0.255963, step = 16529 (146.416 sec)
INFO:tensorflow:global_step/sec: 0.684204
INFO:tensorflow:loss = 0.228186, step = 16629 (146.154 sec)
INFO:tensorflow:global_step/sec: 0.684284
INFO:tensorflow:loss = 0.225595, step = 16729 (146.139 sec)
INFO:tensorflow:global_step/sec: 0.6841
INFO:tensorflow:loss = 0.250814, step = 16829 (146.177 sec)
INFO:tensorflow:Saving checkpoints for 16862 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.683016
INFO:tensorflow:loss = 0.255706, step = 16929 (146.409 sec)
INFO:tensorflow:global_step/sec: 0.68411
INFO:tensorflow:loss = 0.206623, step = 17029 (146.176 sec)
INFO:tensorflow:global_step/sec: 0.684159
INFO:tensorflow:loss = 0.241024, step

INFO:tensorflow:global_step/sec: 0.683979
INFO:tensorflow:loss = 0.214059, step = 22136 (146.204 sec)
INFO:tensorflow:global_step/sec: 0.683898
INFO:tensorflow:loss = 0.170274, step = 22236 (146.220 sec)
INFO:tensorflow:global_step/sec: 0.683961
INFO:tensorflow:loss = 0.178051, step = 22336 (146.208 sec)
INFO:tensorflow:Saving checkpoints for 22413 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.682669
INFO:tensorflow:loss = 0.206459, step = 22436 (146.483 sec)
INFO:tensorflow:global_step/sec: 0.683895
INFO:tensorflow:loss = 0.156226, step = 22536 (146.222 sec)
INFO:tensorflow:global_step/sec: 0.683871
INFO:tensorflow:loss = 0.133318, step = 22636 (146.226 sec)
INFO:tensorflow:global_step/sec: 0.683994
INFO:tensorflow:loss = 0.225863, step = 22736 (146.199 sec)
INFO:tensorflow:Saving checkpoints for 22824 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.682729
INFO:tensorflow:loss = 0.17914, step = 22836 (146.472 sec)
INFO:tensorflow:global_step/sec: 0.

CPU times: user 5min 59s, sys: 27.3 s, total: 6min 26s
Wall time: 6min 21s

[ Epoch 8 ]

Training...

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ubuntu/model/model.ckpt-27349
INFO:tensorflow:Saving checkpoints for 27350 into ubuntu/model/model.ckpt.
INFO:tensorflow:loss = 0.126807, step = 27350
INFO:tensorflow:global_step/sec: 0.683861
INFO:tensorflow:loss = 0.140948, step = 27450 (146.231 sec)
INFO:tensorflow:global_step/sec: 0.684176
INFO:tensorflow:loss = 0.0896522, step = 27550 (146.161 sec)
INFO:tensorflow:global_step/sec: 0.68412
INFO:tensorflow:loss = 0.161047, step = 27650 (146.174 sec)
INFO:tensorflow:global_step/sec: 0.68419
INFO:tensorflow:loss = 0.132904, step = 27750 (146.158 sec)
INFO:tensorflow:Saving checkpoints for 27761 into ubuntu/model/model.ckpt.
INFO:tensorflow:glob

INFO:tensorflow:global_step/sec: 0.684232
INFO:tensorflow:loss = 0.0651491, step = 32857 (146.148 sec)
INFO:tensorflow:Saving checkpoints for 32901 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.683072
INFO:tensorflow:loss = 0.168496, step = 32957 (146.399 sec)
INFO:tensorflow:global_step/sec: 0.6844
INFO:tensorflow:loss = 0.11193, step = 33057 (146.112 sec)
INFO:tensorflow:global_step/sec: 0.684329
INFO:tensorflow:loss = 0.0924525, step = 33157 (146.130 sec)
INFO:tensorflow:global_step/sec: 0.684285
INFO:tensorflow:loss = 0.0790314, step = 33257 (146.137 sec)
INFO:tensorflow:Saving checkpoints for 33312 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.683094
INFO:tensorflow:loss = 0.0802359, step = 33357 (146.393 sec)
INFO:tensorflow:global_step/sec: 0.684166
INFO:tensorflow:loss = 0.103102, step = 33457 (146.163 sec)
INFO:tensorflow:global_step/sec: 0.6842
INFO:tensorflow:loss = 0.0601333, step = 33557 (146.156 sec)
INFO:tensorflow:global_step/sec: 0

INFO:tensorflow:loss = 0.0687184, step = 38564 (146.192 sec)
INFO:tensorflow:global_step/sec: 0.683878
INFO:tensorflow:loss = 0.0539243, step = 38664 (146.225 sec)
INFO:tensorflow:global_step/sec: 0.683901
INFO:tensorflow:loss = 0.0672475, step = 38764 (146.219 sec)
INFO:tensorflow:Saving checkpoints for 38863 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.682759
INFO:tensorflow:loss = 0.0410975, step = 38864 (146.465 sec)
INFO:tensorflow:global_step/sec: 0.684112
INFO:tensorflow:loss = 0.0837657, step = 38964 (146.175 sec)
INFO:tensorflow:global_step/sec: 0.684129
INFO:tensorflow:loss = 0.0862108, step = 39064 (146.172 sec)
INFO:tensorflow:Saving checkpoints for 39070 into ubuntu/model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00396747.
CPU times: user 1h 19min 21s, sys: 18min 8s, total: 1h 37min 29s
Wall time: 1h 35min 16s

Validation...

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tenso

In [12]:
%%time

estimator.evaluate(input_fn_test, name='test')

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Starting evaluation at 2017-06-30-14:38:59
INFO:tensorflow:Restoring parameters from ubuntu/model/model.ckpt-39070
INFO:tensorflow:Finished evaluation at 2017-06-30-14:45:08
INFO:tensorflow:Saving dict for global step 39070: global_step = 39070, loss = 1.8324, recall_at_1_10 = 0.385359408034, recall_at_1_2 = 0.778594080338, recall_at_2_10 = 0.565380549683, recall_at_5_10 = 0.842441860465
CPU times: user 5min 36s, sys: 24.9 s, total: 6min 1s
Wall time: 6min 9s


{'global_step': 39070,
 'loss': 1.8324021,
 'recall_at_1_10': 0.38535940803382662,
 'recall_at_1_2': 0.77859408033826638,
 'recall_at_2_10': 0.56538054968287521,
 'recall_at_5_10': 0.84244186046511627}

** TensorBoard Screenshots**

![Training Loss](Dual LSTM Encoder - TensorBoard - train loss.png)

![Evaluation Loss](Dual LSTM Encoder - TensorBoard - eval loss.png)

![Recall @ 1 (2)](Dual LSTM Encoder - TensorBoard - 1 in 2 R @ 1.png)

![Recall @ 1 (10)](Dual LSTM Encoder - TensorBoard - 1 in 10 R @ 1.png)

![Recall @ 2 (10)](Dual LSTM Encoder - TensorBoard - 1 in 10 R @ 2.png)

![Recall @ 5 (10)](Dual LSTM Encoder - TensorBoard - 1 in 10 R @ 5.png)
